# PODCAST SUMMARIZER USING PYTHON  SCRIPT

Importing libraries


In [2]:
#importing libraries
import pandas as pd
import transformers
import openai
from transformers import BartTokenizer, BartForConditionalGeneration
import os

In [3]:
#specifying file path
file_path = r"116_diarized_timestamped_transcript.txt"